In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from transformers import T5Config, TFT5Model, T5Model, load_tf_weights_in_t5, T5Tokenizer
from transformers import T5ForConditionalGeneration, TFT5ForConditionalGeneration

/home/ubuntu/.local/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [3]:
!rm -rf temp

In [4]:
import os

out = 'temp'
os.makedirs(out, exist_ok=True)

In [5]:
config = T5Config.from_pretrained('malay-huggingface/t5-small-bahasa-cased')
config.save_pretrained(out)

In [6]:
!ls t5-small-noisy-en-ms

checkpoint
events.out.tfevents.1658960344.huseincomel-desktop
graph.pbtxt
model.ckpt-1004200.data-00000-of-00002
model.ckpt-1004200.data-00001-of-00002
model.ckpt-1004200.index
model.ckpt-1004200.meta
model.ckpt-1021200.data-00000-of-00002
model.ckpt-1021200.data-00001-of-00002
model.ckpt-1021200.index
model.ckpt-1021200.meta
model.ckpt-1022200.data-00000-of-00002
model.ckpt-1022200.data-00001-of-00002
model.ckpt-1022200.index
model.ckpt-1022200.meta
model.ckpt-1023200.data-00000-of-00002
model.ckpt-1023200.data-00001-of-00002
model.ckpt-1023200.index
model.ckpt-1023200.meta
model.ckpt-1024200.data-00000-of-00002
model.ckpt-1024200.data-00001-of-00002
model.ckpt-1024200.index
model.ckpt-1024200.meta
model.ckpt-1025200.data-00000-of-00002
model.ckpt-1025200.data-00001-of-00002
model.ckpt-1025200.index
model.ckpt-1025200.meta
model.ckpt-1026200.data-00000-of-00002
model.ckpt-1026200.data-00001-of-00002
model.ckpt-1026200.index
model.ckpt-1026200.meta
model.ckpt-793200.data-00000-of-00002

In [7]:
model = T5Model(config)
load_tf_weights_in_t5(model, config, 't5-small-noisy-en-ms/model.ckpt-1026200')

T5Model(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): FusedRMSNorm(torch.Size([512]), eps=1e-06, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=F

In [8]:
tokenizer = T5Tokenizer('sp10m.cased.ms-en.model', padding = True)

In [9]:
model.save_pretrained(out)

In [10]:
model_gen = T5ForConditionalGeneration.from_pretrained(out)

In [11]:
model_tf = TFT5ForConditionalGeneration.from_pretrained(out, from_pt = True)

2022-08-01 10:50:35.848085: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 10:50:35.880687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 10:50:35.881454: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 10:50:35.882465: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [12]:
with open('en-ms-right.test') as fopen:
    right = fopen.read().split('\n')
    
with open('en-ms-left.test') as fopen:
    left = fopen.read().split('\n')

In [13]:
model_gen.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): FusedRMSNorm(torch.Size([512]), eps=1e-06, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_f

In [14]:
from tqdm import tqdm

batch_size = 1

results = []
for i in tqdm(range(0, len(left), batch_size)):
    input_ids = [{'input_ids': tokenizer.encode(f'terjemah Inggeris ke Melayu: {s}', return_tensors = 'pt')[0]} for s in left[i:i + batch_size]]
    padded = tokenizer.pad(input_ids, padding = 'longest')
    outputs = model_gen.generate(padded['input_ids'].cuda(), attention_mask = padded['attention_mask'].cuda(), max_length = 1000)
    for o in outputs:
        results.append(tokenizer.decode(o, skip_special_tokens=True))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 69744/69744 [5:12:44<00:00,  3.72it/s]


In [16]:
# from tqdm import tqdm

# batch_size = 2

# results = []
# for i in tqdm(range(0, len(left[:1]), batch_size)):
#     input_ids = [{'input_ids': tokenizer.encode(f'terjemah Inggeris ke Melayu: {s}', return_tensors = 'pt')[0]} for s in left[i:i + batch_size]]
#     padded = tokenizer.pad(input_ids, padding = 'longest')
#     outputs = model_gen.generate(**padded, max_length = 1000)
#     for o in outputs:
#         results.append(tokenizer.decode(o, skip_special_tokens=True))

In [17]:
results[0]

'Anda tahu bagaimana kisahnya berjalan. Dua orang kulit putih heteroseksual, hampir pasti berjuang dalam hidup mereka. Mereka bertemu satu sama lain, berkumpul, berpisah, berkumpul lagi dan mencari kebahagiaan dan kesempurnaan yang sebenarnya dalam satu sama lain. Akhirnya, mereka boleh mula menjalani kehidupan yang normal! Rom-coms boleh menjadi pelarian yang menyeronokkan, tetapi ini adalah kisah yang diceritakan terlalu kerap, yang terlalu kekurangan dalam kepelbagaian, terlalu bergantung pada stereotaip gender dan terlalu prihatin dengan menjual jenama cinta yang mustahil untuk hidup: kajian tahun 2008 di Universiti Heriot Watt mendapati bahawa rom-coms mempunyai kesan negatif terhadap hubungan, menjadikan kita mengejar standard cinta yang tidak dapat dicapai. Dalam proses menulis drama baru saya Ross & Rachel, yang menghadapi mitos cinta moden dan dibuka di Edinburgh Festival Fringe pada bulan Ogos ini, saya harus banyak memikirkan percintaan dalam fiksyen dari Romeo dan Juliet hi

In [18]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()

In [19]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [20]:
refs = [filtered_right]
sys = filtered_left

In [21]:
# for i in range(len(filtered_left)):
#     print(filtered_left[i])
#     print(filtered_right[i])
#     print(right[i])
#     print()

In [22]:
r = bleu.corpus_score(sys, refs)

In [23]:
r.__dict__

{'name': 'BLEU',
 'score': 65.00070822235693,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '84.8/70.5/60.6/52.9 (BP = 0.983 ratio = 0.983 hyp_len = 2585122 ref_len = 2630014)',
 'bp': 0.9827843869831318,
 'counts': [2191007, 1773499, 1481652, 1255934],
 'totals': [2585122, 2515379, 2445659, 2376118],
 'sys_len': 2585122,
 'ref_len': 2630014,
 'precisions': [84.75449127739425,
  70.50623385183704,
  60.58293490629724,
  52.85655005349061],
 'prec_str': '84.8/70.5/60.6/52.9',
 'ratio': 0.9829308893412735}

In [24]:
tokenizer.push_to_hub('t5-small-finetuned-noisy-en-ms', organization='mesolitica')

In [25]:
model_gen.push_to_hub('t5-small-finetuned-noisy-en-ms', organization='mesolitica')

Upload file pytorch_model.bin:   0%|          | 4.00k/231M [00:00<?, ?B/s]

To https://huggingface.co/mesolitica/t5-small-finetuned-noisy-en-ms
   b9043ce..091d4f3  main -> main



'https://huggingface.co/mesolitica/t5-small-finetuned-noisy-en-ms/commit/091d4f3b9018f0183c2811f3f8291c0fc2fb575b'

In [26]:
model_tf.push_to_hub('t5-small-finetuned-noisy-en-ms', organization='mesolitica')

Upload file tf_model.h5:   0%|          | 4.00k/231M [00:00<?, ?B/s]

To https://huggingface.co/mesolitica/t5-small-finetuned-noisy-en-ms
   091d4f3..4877d15  main -> main



'https://huggingface.co/mesolitica/t5-small-finetuned-noisy-en-ms/commit/4877d15c5540cc61f8f30403c762a9d74ffd600a'

In [27]:
!cd t5-small-finetuned-noisy-en-ms && git pull

Already up to date.


In [28]:
!cp t5-small-noisy-en-ms/*.tfevents.* t5-small-finetuned-noisy-en-ms
!cd t5-small-finetuned-noisy-en-ms && git add . && git commit -m 'add tensorboard' && git push

[main b16fde5] add tensorboard
 1 file changed, 2 insertions(+), 2 deletions(-)
Uploading LFS objects: 100% (1/1), 68 MB | 6.7 MB/s, done.                      
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 16 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 375 bytes | 375.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
To https://huggingface.co/mesolitica/t5-small-finetuned-noisy-en-ms
   4877d15..b16fde5  main -> main
